In [12]:
import cPickle as pickle
import numpy as np
import tensorflow as tf

In [13]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


In [14]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [15]:
# Make a sub-training set for shorter training time
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # In TensorFlow, all the datasets can be assigned into tf.constant (canstent value tensor)
  # The processing is constructed in form of graph
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [16]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print 'Initialized'
  for step in xrange(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print 'Loss at step', step, ':', l
      print 'Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :])
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print 'Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels)
  print 'Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels)

Initialized
Loss at step 0 : 18.6786
Training accuracy: 9.1%
Validation accuracy: 12.1%
Loss at step 100 : 2.34586
Training accuracy: 71.3%
Validation accuracy: 71.0%
Loss at step 200 : 1.89045
Training accuracy: 74.6%
Validation accuracy: 73.5%
Loss at step 300 : 1.65046
Training accuracy: 75.9%
Validation accuracy: 74.2%
Loss at step 400 : 1.48743
Training accuracy: 76.6%
Validation accuracy: 74.8%
Loss at step 500 : 1.365
Training accuracy: 77.6%
Validation accuracy: 74.9%
Loss at step 600 : 1.26809
Training accuracy: 78.0%
Validation accuracy: 74.9%
Loss at step 700 : 1.18872
Training accuracy: 78.6%
Validation accuracy: 75.2%
Loss at step 800 : 1.12216
Training accuracy: 78.9%
Validation accuracy: 75.3%
Test accuracy: 82.6%


In [17]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [18]:

num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 19.1674
Minibatch accuracy: 10.2%
Validation accuracy: 15.2%
Minibatch loss at step 500 : 1.57437
Minibatch accuracy: 76.6%
Validation accuracy: 75.5%
Minibatch loss at step 1000 : 1.2757
Minibatch accuracy: 77.3%
Validation accuracy: 77.3%
Minibatch loss at step 1500 : 1.29264
Minibatch accuracy: 75.8%
Validation accuracy: 77.4%
Minibatch loss at step 2000 : 1.07499
Minibatch accuracy: 78.9%
Validation accuracy: 77.5%
Minibatch loss at step 2500 : 1.04647
Minibatch accuracy: 82.0%
Validation accuracy: 78.0%
Minibatch loss at step 3000 : 1.1515
Minibatch accuracy: 73.4%
Validation accuracy: 78.4%
Test accuracy: 85.7%


# traing a NN by using stochastic grandient descent 

In [31]:
batch_size = 128
node = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, node]))
  biases = tf.Variable(tf.zeros([node]))
  weights1 = tf.Variable(
    tf.truncated_normal([node,num_labels]))
  biases1 = tf.Variable(tf.zeros([num_labels]))  
  
  # Training computation.
  logits_train = tf.matmul(tf_train_dataset, weights) + biases
  sigmoid_train = tf.nn.relu(logits_train)
  logits_train1 = tf.matmul(sigmoid_train, weights1) + biases1
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits_train1, tf_train_labels))

  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_train1)
  logit_val=tf.matmul(tf_valid_dataset, weights) + biases
  sigmoid_val = tf.nn.relu(logit_val)
  logits_val1 = tf.matmul(sigmoid_val, weights1) + biases1
  valid_prediction = tf.nn.softmax(logits_val1)
  logit_test=tf.matmul(tf_test_dataset, weights) + biases
  sigmoid_test = tf.nn.relu(logit_test)
  logits_test1 = tf.matmul(sigmoid_test, weights1) + biases1
  test_prediction = tf.nn.softmax(logits_test1)

In [32]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 371.728
Minibatch accuracy: 9.4%
Validation accuracy: 32.1%
Minibatch loss at step 500 : 51.1371
Minibatch accuracy: 74.2%
Validation accuracy: 78.9%
Minibatch loss at step 1000 : 6.83205
Minibatch accuracy: 85.2%
Validation accuracy: 79.5%
Minibatch loss at step 1500 : 6.27398
Minibatch accuracy: 80.5%
Validation accuracy: 78.1%
Minibatch loss at step 2000 : 4.25964
Minibatch accuracy: 83.6%
Validation accuracy: 80.6%
Minibatch loss at step 2500 : 8.02424
Minibatch accuracy: 82.8%
Validation accuracy: 81.9%
Minibatch loss at step 3000 : 2.57506
Minibatch accuracy: 83.6%
Validation accuracy: 82.1%
Test accuracy: 89.3%
